# Metabolic Network Reconstruction Tutorial

In the following tutorial you will learn how to manipulate a metabolic model with [cobrapy](https://cobrapy.readthedocs.io/en/latest/) such that you can conform with the recommendations of **Box 2** [in the manuscript on community standards](https://doi.org/10.1101/700112).

From the caption of **Box 2** itself:

> Proposed minimum standardized content for a metabolic network reconstruction. We propose that modelers use this list as a guide to help standardize accessibility, content, and quality; however, more comprehensive documentation and more interpretablee and accessible information can only improve the usability and biological relevance of the shared reconstruction.

Throughout this tutorial we will look at content from the example reconstruction `iPfal19.xml` for _Plasmodium falciparum_ 3D7 provided with the publication, as well as building a minimal example from scratch. Each section will show:

1. Python code needed for the inspection and manipulation of a metabolic reconstruction.
2. Excerpts from the resulting [SBML](http://sbml.org/) that is automatically generated.

In [1]:
import logging

In [2]:
logging.basicConfig(level="INFO")

In [3]:
import cobra
from cobra.io import read_sbml_model

The tutorial was created with the following software versions.

In [4]:
cobra.show_versions()


System Information
OS                    Linux
OS-release 5.0.0-23-generic
Python                3.6.8

Package Versions
cobra                       0.16.0
depinfo                      1.5.1
future                      0.17.1
numpy                       1.17.0
optlang                      1.4.4
pandas                      0.25.0
pip                         19.2.1
python-libsbml-experimental 5.18.0
ruamel.yaml                 0.16.1
setuptools                  41.0.1
six                         1.12.0
swiglpk                     4.65.0
wheel                       0.33.4


## Model

In [5]:
p_falciparum = read_sbml_model("iPfal19.xml")

In [35]:
minimal = cobra.Model("minimal", name="Minimal Demo Model")

Simply saving an empty model with an identifier and name to SBML will generate the following XML element:

```xml
<model metaid="meta_minimal" id="minimal" name="Minimal Demo Model" fbc:strict="true">
```

Please be aware that, by convention, genome-scale metabolic network reconstructions are expected to have at least two compartments: the cytosol and extracellular space. At the moment, support for compartments in cobrapy is a bit weak. You can only create them by referencing them from a metabolite or by reading an existing SBML model. Better support for compartments [is in preparation](https://github.com/opencobra/cobrapy/pull/725).

In their simplest form, compartments are defined as follows in SBML:

```xml
<compartment id="c" name="cytosol" constant="true"/>
```

### Recognized naming convention

# TODO: Recommendation is missing.

### Machine-readable reference to organism and species embedded via MIRIAM annotation

We can find good information about a sequenced species at NCBI. _Plasmodium falciparum_ 3D7 is described [here](https://www.ncbi.nlm.nih.gov/genome/33?genome_assembly_id=369845).

In [8]:
p_falciparum.annotation

{}

In [36]:
minimal.annotation["taxonomy"] = "36329"

This ensures that the reconstruction's taxonomy is annotated. The `annotation` attribute is a Python dictionary whose key-value pairs are automatically converted to MIRIAM compatible URIs. In order for this to work correctly, please first verify the exact spelling of the registry key on [identifiers.org](https://registry.identifiers.org/registry).

```xml
    <annotation>
      <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:vCard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
        <rdf:Description rdf:about="#meta_minimal">
          <bqbiol:is>
            <rdf:Bag>
              <rdf:li rdf:resource="https://identifiers.org/taxonomy/36329"/>
            </rdf:Bag>
          </bqbiol:is>
        </rdf:Description>
      </rdf:RDF>
    </annotation>
```

Important elements here are the description which is about the `model` element due to the `metaid` (compare with section [Model](#Model)).

```xml
<rdf:Description rdf:about="#meta_minimal">
```

The next important element is the biological qualifier

```xml
<bqbiol:is>
```

you can find out more about its meaning at [combine](https://co.mbine.org/standards/qualifiers) and [BioModels](https://www.ebi.ac.uk/biomodels-main/annotation).

Finally, there is one MIRIAM compliant annotation that was generated by cobrapy.

```xml
<rdf:li rdf:resource="https://identifiers.org/taxonomy/36329"/>
```

### NCBI reference genome

The genome assembly is described [here](https://www.ncbi.nlm.nih.gov/assembly/GCF_000002765.4/).

# TODO: It seems to me that identifiers.org does not support GCF identifiers? At least `refseq` registry pattern does not allow `GCF`.

In [37]:
minimal.annotation["insdc.gca"] = "GCA_000002765.2"

Which leads to the following new SBML element:

```xml
<rdf:li rdf:resource="https://identifiers.org/insdc.gca/GCA_000002765.2"/>
```

In [38]:
minimal.annotation["genedb"] = "Plasmodium_falciparum_3D7"

This generates the following familiar element:

```xml
<rdf:li rdf:resource="https://identifiers.org/genedb/Plasmodium_falciparum_3D7"/>
```

### Author(s) contact information embedded

# TODO: Format recommendation is missing.

## Metabolite

A tangent on the term 'metabolite': In the field of metabolic modeling four different terms are often used without clear distinction. There are compounds, chemicals, metabolites, and species. For the purpose of this document, we say 'metabolite' or 'species' to mean the most representative (most common) [tautomer](https://en.wikipedia.org/wiki/Tautomer) at the given pH. Implicitly, this acknowledges that interconvertible groups of tautomers may participate in a reaction. We will use 'compound' or 'chemical' to mean an exact chemical representation only.

An excellent resource for metabolites is [MetaNetX](https://www.metanetx.org/). It merges information from several source databases (among them [KEGG](https://www.genome.jp/kegg/), [ChEBI](https://www.ebi.ac.uk/chebi/), [BiGG](http://bigg.ucsd.edu/)) and aims to provide consistent cross references. At the time of writing there is information on around 700 k compounds in MetaNetX.

For this tutorial, we will look at [1-dodecanoyl-sn-glycerol 3-phosphate](https://www.metanetx.org/chem_info/MNXM4232). On the referenced page you can see that a lot of information that we will need is directly provided for us.

When creating a metabolite identifier, we should be aware of several conventions. As noted before, every metabolite must be allocated to a compartment. Here, we reference the cytosol by its short identifier `"c"`. Since the same metabolite can appear in multiple compartments, it is common practice to add the compartment as a suffix to the identifier of the metabolite. Thus making the identifier unique within the model.

In principle, a metabolite identifier can be any string that satisfies the SBML constraints (which can be expressed by the following regular expression `[a-zA-Z_][a-zA-Z_0-9]*`). However, many modelers choose BiGG identifiers because they often resemble their common names and are thus easier to reason about quickly. We will use this convention here.

In [39]:
metabolite = cobra.Metabolite(id="1ddecg3p_c", compartment="c")

Please note that the metabolite identifier starts with a digit which is against the SBML specification. Luckily for us, cobrapy uses a general `M_` prefix for all metabolites when writing SBML to prevent these cases. In most cases, this is the right default choice but you may prefer different behavior. In those cases you will have to manually adjust the default replacement functions of `cobra.io.write_sbml_model`.

In [40]:
minimal.add_metabolites([metabolite])

When we add the metabolite to the model it produces the following SBML:

```xml
<species id="M_1ddecg3p_c" compartment="c" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
```

### Human readable, descriptive name

The full names are used when displaying more information about a metabolite. This is very helpful when the identifier is rather hard to guess, as is the case for our example, and it will often be the only identifying piece of information that biologists can work with.

In [41]:
metabolite.name = "1-dodecanoyl-sn-glycerol 3-phosphate"

### Charge

The metabolite charge is defined in the SBML package flux-balance constraints (fbc). In most cases the charge should be an integer although the upcoming version 3 of the fbc also allows real numbers for the charge to cover certain edge cases.

In [42]:
metabolite.charge = -2

### Chemical formula

The formula is also an fbc extension attribute.

In [43]:
metabolite.formula = "C15H29O7P"

### InChI strings

The [InChI](https://en.wikipedia.org/wiki/International_Chemical_Identifier) is a very information rich, unique description of a compound. In cobrapy we can provide it as an annotation to the metabolite.

In [44]:
metabolite.annotation["inchi"] = "InChI=1S/C15H31O7P/c1-2-3-4-5-6-7-8-9-10-11-15(17)21-12-14(16)13-22-23(18,19)20/h14,16H,2-13H2,1H3,(H2,18,19,20)/p-2/t14-/m1/s1"

### At least one database identifier from a reliable resource

It might seem annoying and boring work but really: the more the merrier. There are tools that can help you automate this process! (They might also introduce subtle mistakes.) More cross references are better because:

1. There are no one-to-one mappings of identifiers between identifiers and the more you use the better determined your metabolite is.
2. Other users of your model will have data in a myriad of formats. They will thank you deeply, if the identifier namespace of their data already exists in the model.

In [45]:
metabolite.annotation["bigg.metabolite"] = "1ddecg3p"
metabolite.annotation["chebi"] = "CHEBI:62840"
metabolite.annotation["hmdb"] = "HMDB62319"
metabolite.annotation["seed.compound"] = "cpd15325"
metabolite.annotation["metacyc.compound"] = "CPD0-2200"

After adding all of this information, the metabolite SBML definition looks like:

```xml
      <species metaid="meta_M_1ddecg3p_c" id="M_1ddecg3p_c" name="1-dodecanoyl-sn-glycerol 3-phosphate" compartment="c" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false" fbc:charge="-2" fbc:chemicalFormula="C15H29O7P">
        <annotation>
          <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:vCard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
            <rdf:Description rdf:about="#meta_M_1ddecg3p_c">
              <bqbiol:is>
                <rdf:Bag>
                  <rdf:li rdf:resource="https://identifiers.org/inchi/InChI=1S/C15H31O7P/c1-2-3-4-5-6-7-8-9-10-11-15(17)21-12-14(16)13-22-23(18,19)20/h14,16H,2-13H2,1H3,(H2,18,19,20)/p-2/t14-/m1/s1"/>
                  <rdf:li rdf:resource="https://identifiers.org/bigg.metabolite/1ddecg3p"/>
                  <rdf:li rdf:resource="https://identifiers.org/chebi/CHEBI:62840"/>
                  <rdf:li rdf:resource="https://identifiers.org/hmdb/HMDB62319"/>
                  <rdf:li rdf:resource="https://identifiers.org/seed.compound/cpd15325"/>
                  <rdf:li rdf:resource="https://identifiers.org/metacyc.compound/CPD0-2200"/>
                </rdf:Bag>
              </bqbiol:is>
            </rdf:Description>
          </rdf:RDF>
        </annotation>
      </species>
```

## Biochemical reaction

### Human readable, descriptive name

### Reaction formula

### Stoichiometric coefficients

### At least one database identifier from a reliable resource

### EC number

## Gene

### Name and/or identifier

### DNA sequence ID

### Protein sequence ID

### Position (including chromosome, if applicable)

### Associated reactions (GPR)

## Save Model

In [46]:
from cobra.io import write_sbml_model

In [47]:
write_sbml_model(minimal, "minimal.xml")